# Dot Product

The following sequence of cells illustrate hardware processing of a dot product computation:

$$ Z_{m} = A_{k} \times B_{k} $$

<img align="center" src="figures/01.2.1.DUDU_setup.png" alt="DUDU_setup" style="width:70%">

## Exercise 01.2.1 Uncompressed Dense Vectors 

### Understanding the inputs: Problem Specification

In [ ]:
%set_env BASELINE_DIR=../../../example_designs
%set_env DESIGNS_DIR=../../../example_designs/example_designs/sparseloop
%set_env SCRIPTS_DIR=../scripts

In [ ]:
%%bash
cd $DESIGNS_DIR/01.2.1-DUDU-dot-product/ && cat problem.yaml

### Understanding the inputs: Architecture Specification

In [ ]:
%%bash
cd $DESIGNS_DIR/01.2.1-DUDU-dot-product/
cat arch.yaml

### Understanding the inputs: Mapping Specification

In [ ]:
%%bash
cd $DESIGNS_DIR/01.2.1-DUDU-dot-product/
cat mapping.yaml

### Run Example

In [ ]:
%%bash
cd $BASELINE_DIR
python3 run_example_designs.py --architecture sparseloop/01.2.1-DUDU-dot-product

### Examine Important Stats: Storage Capacity & Accesses
Different access types
  - fills: initial writes of the data (zero for 1 level storage, so not shown)
  - reads: streaming out data
  - updates: writebacks of the updated data

In [ ]:
%set_env OUT_DIR=../../../example_designs/example_designs/sparseloop/01.2.1-DUDU-dot-product/outputs/default_problem

In [ ]:
%%bash
export SCRIPT_PATH=$(realpath $SCRIPTS_DIR/01.2.1-DUDU-dot-product-buffer-stats.sh)
chmod +x $SCRIPT_PATH && cd $OUT_DIR && $SCRIPT_PATH

### Examine Important Stats: Computes

In [ ]:
%%bash
export SCRIPT_PATH=$(realpath $SCRIPTS_DIR/01.2.1-DUDU-dot-product-buffer-stats.sh)
chmod +x $SCRIPT_PATH && cd $OUT_DIR && $SCRIPT_PATH

### Examine Important Stats: Summary

In [ ]:
%%bash
export SCRIPT_PATH=$(realpath $SCRIPTS_DIR/01.2.1-DUDU-dot-product-summary-stats.sh)
chmod +x $SCRIPT_PATH && cd $OUT_DIR && $SCRIPT_PATH

## Sparse vector A

<img align="center" src="figures/01.2.2.SUDU_baseline_setup.png" alt="SU_C_DU_setup" style="width:70%">


## Exercise 01.2.2 Sparse Uncompressed  A and Uncompressed Dense B
### Updated Inputs:  Problem Specification Now Reflects Sparsity

In [ ]:
%%bash
cd $DESIGNS_DIR/01.2.2-SUDU-dot-product/ && grep "instance:" problem.yaml -A 10

### Run Example With All Other Inputs Stay the Same

In [ ]:
%%bash
# cd ../designs/01.2.2-SUDU-dot-product
# timeloop-model arch/*.yaml map/*.yaml prob/*.yaml -o output/no-optimization
export S

cd $BASELINE_DIR
python3 run_example_designs.py --architecture sparseloop/01.2.2-SUDU-dot-product --remove-sparse-opts

### Examine Important Stats

You should see **no changes** in runtime statistics. Although there are potential savings introduced by the A vector, with a default uncompressed representation and no sparse optimization applied, it is not possible to exploit such savings. Thus, the baseline architecture's behaviors will not change.

In [ ]:
%set_env OUT_DIR=../../../example_designs/example_designs/sparseloop/01.2.2-SUDU-dot-product/outputs/default_problem

In [ ]:
%%bash
export SCRIPT_PATH=$(realpath $SCRIPTS_DIR/01.2.2-SUDU-dot-product-aggregated-stats.sh)
chmod +x $SCRIPT_PATH && cd $OUT_DIR && $SCRIPT_PATH



### Additional Input:  Sparse Optimization Feature Specification To Enable Gating
- Goal: Perform gating on B and Z based on the payloads in A

<img align="center" src="figures/01.2.2.SUDU_gating_setup.png" alt="figures/01.2.2.SUDU_gating_setup.png" style="width:80%">


In [ ]:
%%bash
# sparse tensor instance
cd $DESIGNS_DIR/01.2.2-SUDU-dot-product/ && cat arch.yaml 

### Run Example With Gating Applied

In [ ]:
%%bash
cd $BASELINE_DIR
python3 run_example_designs.py --architecture sparseloop/01.2.2-SUDU-dot-product

### Examine Important Stats

- `A`: no change, as A tensor is still represented in an uncompressed format and requires the same amount of storage capacity.
- `B`: reduced number of actual reads and **gated reads** counts show up.
- `Z`: reduced number of actual reads and actual updates, and **gated reads** && **gated updates** counts show up.
- `MAC`: reduced number of actual computes. As a result of `gating` on `B`, no operands are sent to `MAC` unit when `A` is empty (i.e., `NOT_EXIST` for both operands). So the number of computes reduced by 75%.
- lower utilization. The reduced number of computes make MAC stays idle for 75% of the cycles, during which there is still access to `A` stored in the `Buffer` (but no accesses to `B` and `Z`). 
- total number of cycles no change.
- total energy reduced.  Reduced number of accesses to `B` and `Z`, and reduced number of computes both save energy 

In [ ]:
%%bash
export SCRIPT_PATH=$(realpath $SCRIPTS_DIR/01.2.2-SUDU-dot-product-aggregated-stats.sh)
chmod +x $SCRIPT_PATH && cd $OUT_DIR && $SCRIPT_PATH

## Exercise 01.2.3 Sparse Compressed A and Dense Uncompressed B


<img align="center" src="figures/01.2.3.SCDU_skipping_setup.png" alt="figures/01.2.3.SCDU_skipping_setup.png" style="width:80%">



### Updated Input: Sparse Optimization Specification
- Represent sparse A in a compressed format: coordinate payload
- Perform skipping based on the compressed A

### Updated Input: Architecture Sepcfication
Extra metadata storage needed to store the *coordiantes* in the compressed representation format

In [ ]:
%%bash
cd $DESIGNS_DIR/01.2.3-SCDU-dot-product
cat arch.yaml

### Run Example with Updated Sparse Optimization Specification and Architecture Specification

In [ ]:
%%bash

# Run example
cd $BASELINE_DIR
python3 run_example_designs.py --architecture sparseloop/01.2.3-SCDU-dot-product

### Examine Important Stats

- `A` related:
  - Reduced capacity requirement and reduced number of accesses due to compressed data representation format.
  - Extra metadata storage overhead, extra metadata access overhead. Note that # of metadata units equals to # of nonzero data in A.
- `B`: Reduced number of actual reads and **skipped reads** counts show up due to explicit skipping optimization.
- `Z`: Reduced number of actual reads and actual updates, and **skipped reads** && **skipped updates** counts show up due to explicit skipping optimization.
- `MAC`: Reduced number of actual computes. As a result of `skipping` on `B`, no operands are sent to `MAC` unit when `A` is empty (i.e., `NOT_EXIST` for both operands). So the number of computes reduced by 75%.
- Full utilization at 1.0. Since both computes and storage accesses are skipped, the utilization of the MAC unit is still full and the total number of cycles reduced as well.
- Total number of cycles reduced.
- Total energy reduced.

In [ ]:
%set_env OUT_DIR=../../../example_designs/example_designs/sparseloop/01.2.3-SCDU-dot-product/outputs/default_problem

In [ ]:
%%bash
export SCRIPT_PATH=$(realpath $SCRIPTS_DIR/01.2.3-SCDU-dot-product-aggregated-stats.sh)
chmod +x $SCRIPT_PATH && cd $OUT_DIR && $SCRIPT_PATH